# 🚙 Dynamic Pricing for Urban Parking Lots
   ## Overview
This project implements a real-time dynamic pricing system for urban parking lots using:
*   Pathway for streaming data ingestion and processing
*   Pandas for preprocessing and cleaning
* A demand-based pricing model incorporating multiple factors
* Output aggregation in daily
The goal is to simulate how parking prices can dynamically adapt to occupancy, queue lengths, traffic, vehicle type, and special days.

## Objectives

* Preprocess historical parking data and clean inconsistencies
* Simulate a streaming data pipeline
* Compute daily demand and prices for each parking lot
* Export results for analysis


# Step 0-Install Dependencies\
Installing the required libraries and dependencies.

In [ ]:
!pip install pathway bokeh panel --quiet


In [2]:
import pandas as pd
import numpy as np
import pathway as pw
import datetime
import panel as pn
import bokeh.plotting


# Importing and Preprocessing the Data
## Rationale
The original dataset contained:

* Timestamps split across two columns

* Textual labels in numeric fields (e.g., "low", "high", "average")

* Potential mixed types

Cleaning was essential to avoid parse errors during ingestion.

##Actions
* Combined LastUpdatedDate and LastUpdatedTime into a single Timestamp

* Mapped VehicleType (e.g., car, truck) to numeric weights

* Replaced text labels with numeric equivalents:

 * "low" → 0.2

 * "medium" → 0.5

 * "high" → 0.8

 * "average" → 0.5

* Converted all numeric columns (Occupancy, Capacity, QueueLength, TrafficLevel, IsSpecialDay, VehicleTypeWeight) to numeric types

* Filled any missing values with 0

##Outcome: Clean, sorted CSV ready for streaming.

In [3]:
# Load your raw dataset
df_raw = pd.read_csv('/content/dataset.csv')

# Combine date and time to make Timestamp
df_raw["Timestamp"] = pd.to_datetime(
    df_raw["LastUpdatedDate"] + " " + df_raw["LastUpdatedTime"],
    format="%d-%m-%Y %H:%M:%S"
)

# Rename column
df_raw = df_raw.rename(columns={"TrafficConditionNearby": "TrafficLevel"})

# Encode VehicleType
vehicle_type_map = {
    "car": 1.0,
    "bike": 0.7,
    "truck": 1.3
}
df_raw["VehicleTypeWeight"] = df_raw["VehicleType"].map(vehicle_type_map)

# Reorder and clean numeric columns
df_clean = pd.DataFrame({
    "Timestamp": df_raw["Timestamp"],
    "SystemCodeNumber": df_raw["SystemCodeNumber"],
    "Occupancy": pd.to_numeric(df_raw["Occupancy"], errors="coerce").fillna(0),
    "Capacity": pd.to_numeric(df_raw["Capacity"], errors="coerce").fillna(0),
    "QueueLength": pd.to_numeric(df_raw["QueueLength"], errors="coerce").fillna(0),
    "TrafficLevel": df_raw["TrafficLevel"].replace({
        "low": 0.2,
        "medium": 0.5,
        "high": 0.8,
        "low.in": 0.2,
        "high.in": 0.8,
        "average": 0.5
    }),
    "IsSpecialDay": pd.to_numeric(df_raw["IsSpecialDay"], errors="coerce").fillna(0),
    "VehicleTypeWeight": pd.to_numeric(df_raw["VehicleTypeWeight"], errors="coerce").fillna(0)
})

# Ensure TrafficLevel numeric
df_clean["TrafficLevel"] = pd.to_numeric(df_clean["TrafficLevel"], errors="coerce").fillna(0)

# Sort rows
df_clean = df_clean.sort_values(["SystemCodeNumber", "Timestamp"]).reset_index(drop=True)

# Save cleaned CSV
df_clean.to_csv("parking_stream.csv", index=False)

print("✅ Cleaned CSV saved.")


✅ Cleaned CSV saved.


/tmp/ipython-input-3-1039630409.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "TrafficLevel": df_raw["TrafficLevel"].replace({


#step 1-Define the Pathway Schema
Pathway requires a strict schema for ingestion. Each column must have a consistent type.

In [4]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: float
    IsSpecialDay: int
    VehicleTypeWeight: float


#step 2-Load Data as a Real-Time Stream
simulate the data streaming into your pricing engine.

In [5]:
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=500
)
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


#step 3-Demand-Based Pricing Logic
 1.Parameters

* Feel free to tweak these to adjust price sensitivity.

In [6]:
ALPHA = 2.0
BETA = 0.5
GAMMA = 1.0
DELTA = 1.5
EPSILON = 1.0
LAMBDA = 1.0
BASE_PRICE = 10.0


#step 4-Compute Daily Demand and Prices
Use daily tumbling windows per parking lot.

In [7]:
import datetime

# Daily window
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber + "_" + pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_avg = pw.reducers.avg(pw.this.Occupancy),
        queue_avg = pw.reducers.avg(pw.this.QueueLength),
        traffic_avg = pw.reducers.avg(pw.this.TrafficLevel),
        special = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_avg = pw.reducers.avg(pw.this.VehicleTypeWeight),
        cap = pw.reducers.max(pw.this.Capacity),
        lot = pw.reducers.max(pw.this.SystemCodeNumber)
    )
)

# Compute demand step
delta_with_demand = delta_window.with_columns(
    demand = (
        ALPHA * (pw.this.occ_avg / pw.this.cap)
        + BETA * pw.this.queue_avg
        - GAMMA * pw.this.traffic_avg
        + DELTA * pw.this.special
        + EPSILON * pw.this.vehicle_avg
    )
)

# Compute normalized demand
delta_with_norm = delta_with_demand.with_columns(
    norm_demand = pw.this.demand / 5.0
)

# Compute raw price
delta_with_raw = delta_with_norm.with_columns(
    raw_price = BASE_PRICE * (1 + LAMBDA * pw.this.norm_demand)
)

# Compute bounded price
delta_with_price = delta_with_raw.with_columns(
    price = pw.if_else(
        pw.this.raw_price < BASE_PRICE * 0.5,
        BASE_PRICE * 0.5,
        pw.if_else(
            pw.this.raw_price > BASE_PRICE * 2.0,
            BASE_PRICE * 2.0,
            pw.this.raw_price
        )
    )
)


#step 5-Visualization with a Bokeh.
Note: The Bokeh plot in the next cell will only be generated after you run the pw.run() cell (i.e., the final cell).

In [8]:
# Activate Panel
pn.extension()

# Bokeh plotting function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Daily Dynamic Prices per Parking Lot",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

# Bind the visualization
viz = delta_with_price.plot(price_plotter, sorting_col="t")

# Make it servable as a Panel app
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

###step 6- this is the step i was getting problem so i took a little help from chatgpt and fix it, the error was just a simple task we had high somewehre in the code so we just convert everything to numeric.

In [9]:
import pandas as pd

# Reload CSV
df_clean = pd.read_csv("parking_stream.csv")

# Convert all columns to string to avoid mixed types
df_clean = df_clean.astype(str)

# List all numeric columns in your schema
numeric_cols = [
    "Occupancy",
    "Capacity",
    "QueueLength",
    "TrafficLevel",
    "IsSpecialDay",
    "VehicleTypeWeight"
]

# Replace common text labels
replacements = {
    "low": 0.2,
    "medium": 0.5,
    "high": 0.8,
    "low.in": 0.2,
    "high.in": 0.8,
    "average": 0.5
}

for col in numeric_cols:
    df_clean[col] = df_clean[col].replace(replacements)

# Convert all numeric columns to numbers
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce").fillna(0)

# Show unique values for verification
print("✅ Cleaned unique values:")
for col in numeric_cols:
    print(f"{col}:", df_clean[col].unique())

# Save back to CSV
df_clean.to_csv("parking_stream.csv", index=False)

print("✅ Data re-cleaned and saved.")


✅ Cleaned unique values:
Occupancy: [  61   64   80 ... 1463 1510 1487]
Capacity: [ 577  387  470  687 1200  485  690 2009 2803 3883 2937 1322 3103 1920]
QueueLength: [ 1  2  3  6  5  8  7  4  9 10  0 11 12 13 14 15]
TrafficLevel: [0.2 0.8 0.5]
IsSpecialDay: [0 1]
VehicleTypeWeight: [1.  0.7 1.3 0. ]
✅ Data re-cleaned and saved.


#step 7- running the pathway.....

In [10]:
%%capture --no-display
pw.run()


Output()